In [42]:
import cv2
import numpy as np
import pandas as pd
import os

In [65]:
path='/Users/krishrana/Python/julia/train_2'
img_size=64
training_data=[]

for i in range(1,6284):
    #print(os.path.join(path, str(i)+'.Bmp'))
    img_array=cv2.imread(os.path.join(path, str(i)+'.BMP'))
    new_array=cv2.resize(img_array, (img_size, img_size))
    
    training_data.append(new_array)

print(len(training_data))

6283


In [66]:
df=pd.read_csv('/Users/krishrana/Python/julia/trainLabels.csv')
labels=df.iloc[:, 1]
labels.head()

0    n
1    8
2    T
3    I
4    R
Name: Class, dtype: object

In [67]:
from sklearn import preprocessing
labels=np.array(labels)
le = preprocessing.LabelEncoder()
int_labels=le.fit_transform(labels)
print(int_labels)

[49  8 29 ... 25 23 27]


In [69]:
training_data=np.array(training_data)
training_data=training_data.reshape(6283, 64, 64, 3)
print(training_data.shape)

(6283, 64, 64, 3)


In [70]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from keras import optimizers

In [81]:
model=Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(training_data.shape[1:])))
#model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), activation='relu'))
#model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128, (3,3), activation='relu'))
#model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(62))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_104 (Conv2D)          (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_56 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_105 (Conv2D)          (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_57 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_106 (Conv2D)          (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_58 (MaxPooling (None, 6, 6, 128)         0         
_________________________________________________________________
flatten_15 (Flatten)         (None, 4608)            

In [82]:
adam=optimizers.Adam(lr=0.001)
batch_size=16
epochs=30

In [83]:
training_data=training_data/255
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
target = to_categorical(int_labels, num_classes=62)
print(target[:5, :])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [84]:

model.fit(training_data, target, epochs=epochs, batch_size=batch_size, validation_split=0.2)

Train on 5026 samples, validate on 1257 samples
Epoch 1/30
5026/5026 [==============================] - 22s 4ms/sample - loss: 3.8489 - acc: 0.0641 - val_loss: 3.7656 - val_acc: 0.0724
Epoch 2/30
5026/5026 [==============================] - 23s 5ms/sample - loss: 3.5835 - acc: 0.1315 - val_loss: 2.8419 - val_acc: 0.3270
Epoch 3/30
5026/5026 [==============================] - 26s 5ms/sample - loss: 2.4228 - acc: 0.3846 - val_loss: 1.9591 - val_acc: 0.5028
Epoch 4/30
5026/5026 [==============================] - 26s 5ms/sample - loss: 1.7028 - acc: 0.5446 - val_loss: 1.6004 - val_acc: 0.5815
Epoch 5/30
5026/5026 [==============================] - 29s 6ms/sample - loss: 1.3037 - acc: 0.6251 - val_loss: 1.3373 - val_acc: 0.6396
Epoch 6/30
5026/5026 [==============================] - 37s 7ms/sample - loss: 1.0652 - acc: 0.6858 - val_loss: 1.2806 - val_acc: 0.6587
Epoch 7/30
5026/5026 [==============================] - 42s 8ms/sample - loss: 0.9122 - acc: 0.7203 - val_loss: 1.2201 - val_acc: 

In [85]:
model.save('streetNet.pt')

In [98]:
path_test='/Users/krishrana/Python/julia/test'
img_size=64
test_data=[]

for i in range(6284, 12504):
    #print(os.path.join(path_test, str(i)+'.Bmp'))
    img_array=cv2.imread(os.path.join(path_test, str(i)+'.BMP'))
    new_array=cv2.resize(img_array, (img_size, img_size))
    
    test_data.append(new_array)

print(len(test_data))

6220


In [109]:
test_data=np.array(test_data)
print(test_data.shape)

(6220, 64, 64, 3)


In [115]:
pred=model.predict(test_data)

In [119]:
y = np.argmax(pred, axis=-1)
print(y)
int_target=le.inverse_transform(y)
predections=pd.DataFrame(int_target, columns=['class']).to_csv('street.csv')

[17 14 35 ... 23 23 22]
